# 리뷰
## 리뷰 평점과 구매의 상관관계

In [1]:
import pandas as pd
import pymysql
import sqlalchemy
from urllib import parse

user = 'hyun'
password = 'hyun'
host='172.20.224.1'
port = 3306
database = 'KPI'
password = parse.quote_plus(password)
engine = sqlalchemy.create_engine(f"mysql://{user}:{password}@{host}:{port}/{database}")

In [2]:
customers = pd.read_sql_query("SELECT * FROM customers", con=engine)
locations = pd.read_sql_query("SELECT * FROM locations", con=engine)
order_items = pd.read_sql_query("SELECT * FROM order_items", con=engine)
orders = pd.read_sql_query("SELECT * FROM orders", con=engine)
payments = pd.read_sql_query("SELECT * FROM payments", con=engine)
products = pd.read_sql_query("SELECT * FROM products", con=engine)
reviews = pd.read_sql_query("SELECT * FROM reviews", con=engine)
sellers = pd.read_sql_query("SELECT * FROM sellers", con=engine)
order_items_category = pd.read_sql_query("SELECT * FROM order_items_category", con=engine)

### 리뷰 아이디에 중복값 확인하려고 ..

In [3]:
reviews

,Review_id,Order_id,Review_score,Review_creation_date,Review_answer_timestamp
0,REVIEW_00000,ORDER_01674,4,2019-01-18 00:00:00,2019-01-18 21:46:59
1,REVIEW_00001,ORDER_80140,5,2019-03-10 00:00:00,2019-03-11 03:05:13
2,REVIEW_00002,ORDER_69816,5,2019-02-17 00:00:00,2019-02-18 14:36:24
3,REVIEW_00003,ORDER_24398,5,2018-04-21 00:00:00,2018-04-21 22:02:06
4,REVIEW_00004,ORDER_70366,5,2019-03-01 00:00:00,2019-03-02 10:26:53
...,...,...,...,...,...
87868,REVIEW_87298,ORDER_58840,5,2019-07-07 00:00:00,2019-07-14 17:18:30
87869,REVIEW_87299,ORDER_75162,5,2018-12-09 00:00:00,2018-12-11 20:06:42
87870,REVIEW_87300,ORDER_08690,5,2019-03-22 00:00:00,2019-03-23 09:10:43
87871,REVIEW_87301,ORDER_25681,4,2019-07-01 00:00:00,2019-07-02 12:59:13


In [22]:
reviews['Review_id'].unique()

array(['REVIEW_00000', 'REVIEW_00001', 'REVIEW_00002', ...,
       'REVIEW_87300', 'REVIEW_87301', 'REVIEW_87302'], dtype=object)

In [14]:
# 중복값 숫자 파악
reviews.duplicated(['Review_id']).value_counts()

False    87173
True       700
Name: count, dtype: int64

In [157]:
df = reviews[reviews.duplicated(['Order_id'])]

In [158]:
# 리뷰즈에서 오더 아이디 중복인 애들 뽑아서 보기
df

,Review_id,Order_id,Review_score,Review_creation_date,Review_answer_timestamp
1001,REVIEW_01002,ORDER_08418,5,2018-11-07 00:00:00,2018-11-10 20:07:48
2767,REVIEW_02773,ORDER_68723,5,2019-01-18 00:00:00,2019-01-18 17:36:45
7216,REVIEW_07224,ORDER_85111,5,2019-03-23 00:00:00,2019-03-24 00:23:06
8059,REVIEW_08068,ORDER_04544,5,2018-10-21 00:00:00,2018-10-22 00:47:25
8712,REVIEW_08722,ORDER_14054,3,2018-09-14 00:00:00,2018-09-17 11:14:08
...,...,...,...,...,...
87473,REVIEW_86910,ORDER_40011,5,2019-04-03 00:00:00,2019-04-10 14:38:27
87667,REVIEW_53847,ORDER_39488,5,2019-02-17 00:00:00,2019-02-19 19:29:19
87768,REVIEW_06134,ORDER_09043,4,2018-07-03 00:00:00,2018-07-05 19:06:59
87817,REVIEW_37828,ORDER_44549,5,2019-03-03 00:00:00,2019-03-04 22:56:47


In [160]:
df.loc[df['Order_id']=='ORDER_09043']

,Review_id,Order_id,Review_score,Review_creation_date,Review_answer_timestamp
87768,REVIEW_06134,ORDER_09043,4,2018-07-03 00:00:00,2018-07-05 19:06:59


In [47]:
# 리뷰즈에서 오더 아이디 중복인 애들 중에서 오더 아이디 겹치면 합치기
df2 = df.groupby(['Order_id']).sum().reset_index()

In [45]:
df2.loc[df2['Order_id']=='ORDER_00354']

,Order_id,Review_id,Review_score,Review_creation_date,Review_answer_timestamp
0,ORDER_00354,REVIEW_32316,1,2019-03-23 00:00:00,2019-03-24 12:57:14


In [53]:
df2

,Order_id,Review_id,Review_score,Review_creation_date,Review_answer_timestamp
0,ORDER_00354,REVIEW_32316,1,2019-03-23 00:00:00,2019-03-24 12:57:14
1,ORDER_00486,REVIEW_63855,5,2019-01-27 00:00:00,2019-01-30 19:18:27
2,ORDER_00589,REVIEW_65703,5,2018-11-25 00:00:00,2018-11-25 21:21:54
3,ORDER_00717,REVIEW_19068,1,2018-04-12 00:00:00,2018-04-13 01:14:56
4,ORDER_00971,REVIEW_55195,1,2018-12-14 00:00:00,2018-12-14 13:23:36
...,...,...,...,...,...
515,ORDER_87558,REVIEW_53289,4,2019-05-10 00:00:00,2019-05-11 10:52:52
516,ORDER_87718,REVIEW_50667,5,2018-06-25 00:00:00,2018-07-01 11:39:23
517,ORDER_87741,REVIEW_23082,5,2018-12-15 00:00:00,2018-12-16 06:50:19
518,ORDER_87769,REVIEW_71275,5,2019-01-23 00:00:00,2019-01-24 00:17:19


In [15]:
reviews.duplicated(['Order_id']).value_counts()

False    87349
True       524
Name: count, dtype: int64

In [52]:
df2.loc[df2['Review_score'] >= 5]

,Order_id,Review_id,Review_score,Review_creation_date,Review_answer_timestamp
1,ORDER_00486,REVIEW_63855,5,2019-01-27 00:00:00,2019-01-30 19:18:27
2,ORDER_00589,REVIEW_65703,5,2018-11-25 00:00:00,2018-11-25 21:21:54
7,ORDER_01460,REVIEW_73303,5,2019-02-27 00:00:00,2019-03-02 00:14:40
8,ORDER_01532,REVIEW_47205,5,2018-06-28 00:00:00,2018-06-29 02:57:20
9,ORDER_01842,REVIEW_83633,5,2018-09-13 00:00:00,2018-09-14 08:23:06
...,...,...,...,...,...
513,ORDER_87084,REVIEW_06691,5,2018-02-15 00:00:00,2018-02-21 23:30:22
516,ORDER_87718,REVIEW_50667,5,2018-06-25 00:00:00,2018-07-01 11:39:23
517,ORDER_87741,REVIEW_23082,5,2018-12-15 00:00:00,2018-12-16 06:50:19
518,ORDER_87769,REVIEW_71275,5,2019-01-23 00:00:00,2019-01-24 00:17:19


In [43]:
reviews.loc[reviews['Order_id']=='ORDER_00354']

,Review_id,Order_id,Review_score,Review_creation_date,Review_answer_timestamp
21657,REVIEW_16414,ORDER_00354,1,2019-03-25 00:00:00,2019-03-25 21:40:11
78458,REVIEW_32316,ORDER_00354,1,2019-03-23 00:00:00,2019-03-24 12:57:14


같은 주문(제품)에 대해 리뷰를 2번 쓴 경우가 있다..

중복값... 

In [20]:
order_items_category.duplicated(['Order_id']).value_counts()

False    87955
True     12643
Name: count, dtype: int64

In [28]:
order_items_category.duplicated(['Order_id']).duplicated()

0         False
1          True
2          True
3          True
4          True
          ...  
100593     True
100594     True
100595     True
100596     True
100597     True
Length: 100598, dtype: bool

reviews 파일 봤을 때 어떤 제품에 대한 리뷰인지 확인이 불가능 하므로,  
오더 아이디를 통해서 제품 아이디와 해당 제품이 속한 카테고리를 가져오고자 함

In [29]:
order_items_category[order_items_category.duplicated(['Order_id'])]

,Order_id,Order_item_id,Product_id,Seller_id,Price,Freight_value,Product_category_name
5,ORDER_63118,1.0,PRODUCT_00042,SELLER_0556,39.90,17.60,furniture_decor
10,ORDER_82326,3.0,PRODUCT_00148,SELLER_0697,54.90,23.31,home_utilities
11,ORDER_82326,2.0,PRODUCT_00148,SELLER_0697,54.90,23.31,home_utilities
12,ORDER_82326,1.0,PRODUCT_00148,SELLER_0697,54.90,23.31,home_utilities
27,ORDER_84801,1.0,PRODUCT_00231,SELLER_1049,72.00,21.58,electronics
...,...,...,...,...,...,...,...
100553,ORDER_13349,2.0,PRODUCT_27815,SELLER_0533,29.99,16.79,telephony
100566,ORDER_23038,3.0,PRODUCT_28359,SELLER_0042,119.00,4.40,bed_bath_table
100570,ORDER_04606,1.0,PRODUCT_28501,SELLER_2194,76.90,28.94,home_comfort
100572,ORDER_75173,2.0,PRODUCT_28501,SELLER_2194,76.90,12.04,home_comfort


망할 놈들이 오더 아이디도 겹치고 한 오더에 여러 개 주문한 걸 표를 다 나눠서 함

In [25]:
order_items_category.loc[order_items_category['Order_id']=='ORDER_63118']

,Order_id,Order_item_id,Product_id,Seller_id,Price,Freight_value,Product_category_name
4,ORDER_63118,2.0,PRODUCT_00042,SELLER_0556,39.9,17.6,furniture_decor
5,ORDER_63118,1.0,PRODUCT_00042,SELLER_0556,39.9,17.6,furniture_decor


일단은 같은 오더 아이디에서는 같은 제품을 중목으로 구매한 것만 있다고 판단이 됨 ..  
그러니 이를 합치는 것이 옳을 듯..?

In [54]:
orders

,Order_id,Customer_id,Order_status,Order_purchase_timestamp,Order_delivered_carrier_date,Order_delivered_customer_date,Order_estimated_delivery_date
0,ORDER_00000,CUSTOMER_62232,delivered,2018-10-02 10:56:33,2018-10-04 19:55:00,2018-10-10 21:25:13,2018-10-18
1,ORDER_00001,CUSTOMER_54059,delivered,2018-11-18 19:28:06,2018-11-22 13:39:59,2018-12-02 00:28:42,2018-12-15
2,ORDER_00002,CUSTOMER_59535,delivered,2019-02-13 21:18:39,2019-02-14 19:46:34,2019-02-16 18:17:02,2019-02-26
3,ORDER_00003,CUSTOMER_60516,delivered,2018-07-09 21:57:05,2018-07-11 14:58:04,2018-07-26 10:57:55,2018-08-01
4,ORDER_00004,CUSTOMER_76957,delivered,2018-05-16 13:10:30,2018-05-22 10:07:46,2018-05-26 12:55:51,2018-06-07
...,...,...,...,...,...,...,...
87950,ORDER_88083,CUSTOMER_53611,delivered,2018-03-09 09:54:05,2018-03-10 11:18:03,2018-03-17 15:08:01,2018-03-28
87951,ORDER_88084,CUSTOMER_69438,delivered,2019-02-06 12:58:58,2019-02-07 23:22:42,2019-02-28 17:37:56,2019-03-02
87952,ORDER_88085,CUSTOMER_17637,delivered,2018-08-27 14:46:43,2018-08-28 20:52:26,2018-09-21 11:24:17,2018-09-27
87953,ORDER_88086,CUSTOMER_15001,delivered,2019-01-08 21:28:27,2019-01-12 15:35:03,2019-01-25 23:32:54,2019-02-15


In [56]:
orders['Order_id'].duplicated().value_counts()

Order_id
False    87955
Name: count, dtype: int64

# 아 오더스에서 희망을 찾았다  
# 오더스로 일단 우겨본다
# 멸망

In [57]:
order_items_category['Order_id'].duplicated().value_counts()

Order_id
False    87955
True     12643
Name: count, dtype: int64

In [67]:
# 오더 아이디 겹치는 거 뽑음
order_items_category[order_items_category.duplicated(['Order_id'])]

,Order_id,Order_item_id,Product_id,Seller_id,Price,Freight_value,Product_category_name
5,ORDER_63118,1.0,PRODUCT_00042,SELLER_0556,39.90,17.60,furniture_decor
10,ORDER_82326,3.0,PRODUCT_00148,SELLER_0697,54.90,23.31,home_utilities
11,ORDER_82326,2.0,PRODUCT_00148,SELLER_0697,54.90,23.31,home_utilities
12,ORDER_82326,1.0,PRODUCT_00148,SELLER_0697,54.90,23.31,home_utilities
27,ORDER_84801,1.0,PRODUCT_00231,SELLER_1049,72.00,21.58,electronics
...,...,...,...,...,...,...,...
100553,ORDER_13349,2.0,PRODUCT_27815,SELLER_0533,29.99,16.79,telephony
100566,ORDER_23038,3.0,PRODUCT_28359,SELLER_0042,119.00,4.40,bed_bath_table
100570,ORDER_04606,1.0,PRODUCT_28501,SELLER_2194,76.90,28.94,home_comfort
100572,ORDER_75173,2.0,PRODUCT_28501,SELLER_2194,76.90,12.04,home_comfort


In [70]:
order_items_category.loc[order_items_category['Order_id']=='ORDER_82326']

,Order_id,Order_item_id,Product_id,Seller_id,Price,Freight_value,Product_category_name
9,ORDER_82326,4.0,PRODUCT_00148,SELLER_0697,54.9,23.31,home_utilities
10,ORDER_82326,3.0,PRODUCT_00148,SELLER_0697,54.9,23.31,home_utilities
11,ORDER_82326,2.0,PRODUCT_00148,SELLER_0697,54.9,23.31,home_utilities
12,ORDER_82326,1.0,PRODUCT_00148,SELLER_0697,54.9,23.31,home_utilities


In [71]:
reviews

,Review_id,Order_id,Review_score,Review_creation_date,Review_answer_timestamp
0,REVIEW_00000,ORDER_01674,4,2019-01-18 00:00:00,2019-01-18 21:46:59
1,REVIEW_00001,ORDER_80140,5,2019-03-10 00:00:00,2019-03-11 03:05:13
2,REVIEW_00002,ORDER_69816,5,2019-02-17 00:00:00,2019-02-18 14:36:24
3,REVIEW_00003,ORDER_24398,5,2018-04-21 00:00:00,2018-04-21 22:02:06
4,REVIEW_00004,ORDER_70366,5,2019-03-01 00:00:00,2019-03-02 10:26:53
...,...,...,...,...,...
87868,REVIEW_87298,ORDER_58840,5,2019-07-07 00:00:00,2019-07-14 17:18:30
87869,REVIEW_87299,ORDER_75162,5,2018-12-09 00:00:00,2018-12-11 20:06:42
87870,REVIEW_87300,ORDER_08690,5,2019-03-22 00:00:00,2019-03-23 09:10:43
87871,REVIEW_87301,ORDER_25681,4,2019-07-01 00:00:00,2019-07-02 12:59:13


In [72]:
order_items

,Order_id,Order_item_id,Product_id,Seller_id,Price,Freight_value
0,ORDER_75477,1,PRODUCT_23128,SELLER_0452,58.90,13.29
1,ORDER_63604,1,PRODUCT_24351,SELLER_0414,239.90,19.93
2,ORDER_05566,1,PRODUCT_20226,SELLER_1637,199.00,17.87
3,ORDER_04632,1,PRODUCT_07878,SELLER_1445,199.90,18.14
4,ORDER_61452,1,PRODUCT_03505,SELLER_0581,21.90,12.69
...,...,...,...,...,...,...
100552,ORDER_70405,1,PRODUCT_04209,SELLER_2078,299.99,43.41
100553,ORDER_62069,1,PRODUCT_08229,SELLER_1593,350.00,36.53
100554,ORDER_46632,1,PRODUCT_21462,SELLER_0528,99.90,16.95
100555,ORDER_52966,1,PRODUCT_19978,SELLER_2563,55.99,8.72


In [91]:
order_items[order_items.duplicated(['Order_id'])]

,Order_id,Order_item_id,Product_id,Seller_id,Price,Freight_value
12,ORDER_47740,2,PRODUCT_04082,SELLER_0417,49.90,13.37
30,ORDER_32079,2,PRODUCT_19082,SELLER_0199,21.33,15.10
31,ORDER_32079,3,PRODUCT_19082,SELLER_0199,21.33,15.10
40,ORDER_23881,2,PRODUCT_23679,SELLER_2076,24.89,17.63
41,ORDER_23881,3,PRODUCT_23679,SELLER_2076,24.89,17.63
...,...,...,...,...,...,...
100525,ORDER_06151,3,PRODUCT_14791,SELLER_0290,21.90,15.79
100542,ORDER_75386,2,PRODUCT_14224,SELLER_1501,180.00,48.14
100548,ORDER_54839,2,PRODUCT_06372,SELLER_2054,55.00,34.19
100549,ORDER_54839,3,PRODUCT_06372,SELLER_2054,55.00,34.19


In [94]:
df=order_items.groupby(['Order_id', 'Product_id']).sum().reset_index()
# 이게 오더 아이템즈에서 오더 아이디랑 포로덕트 아이디 동일한 애들 합친 것

In [92]:
df.loc[df['Order_id']=='ORDER_47740']

,Order_id,Product_id,Order_item_id,Seller_id,Price,Freight_value
49429,ORDER_47740,PRODUCT_04082,3,SELLER_0417SELLER_0417,99.8,26.74


In [93]:
order_items.loc[order_items['Order_id']=='ORDER_47740']

,Order_id,Order_item_id,Product_id,Seller_id,Price,Freight_value
11,ORDER_47740,1,PRODUCT_04082,SELLER_0417,49.9,13.37
12,ORDER_47740,2,PRODUCT_04082,SELLER_0417,49.9,13.37


In [95]:
df[df.duplicated(['Order_id'])]

,Order_id,Product_id,Order_item_id,Seller_id,Price,Freight_value
23,ORDER_00022,PRODUCT_12896,2,SELLER_0823,39.50,48.64
58,ORDER_00056,PRODUCT_07372,1,SELLER_2445,119.00,24.97
108,ORDER_00105,PRODUCT_14388,2,SELLER_1059,119.99,28.05
109,ORDER_00105,PRODUCT_16209,1,SELLER_1678,179.90,8.41
124,ORDER_00119,PRODUCT_23470,5,SELLER_0613SELLER_0613,183.98,27.00
...,...,...,...,...,...,...
91092,ORDER_87885,PRODUCT_11016,2,SELLER_1436,180.00,0.29
91119,ORDER_87912,PRODUCT_10235,1,SELLER_0062,209.99,15.90
91183,ORDER_87975,PRODUCT_28697,13,SELLER_0417SELLER_0417SELLER_0417SELLER_0417,199.60,97.48
91231,ORDER_88022,PRODUCT_16854,1,SELLER_0730,28.50,5.01


In [96]:
# 오더 아이디가 같은데 셀러가 여럿인 경우가 있네 ,,

In [104]:
# 그래서 오더 아이템즈에서 해당 오더 아이디 검색해보니~
# 한 오더 아이디에 서로 다른 제품 담은 걸 봄 ㅎㅎ ;

In [101]:
order_items.loc[order_items['Order_id']=='ORDER_87975']

,Order_id,Order_item_id,Product_id,Seller_id,Price,Freight_value
97896,ORDER_87975,1,PRODUCT_28697,SELLER_0417,49.9,24.37
97897,ORDER_87975,2,PRODUCT_12522,SELLER_0417,49.9,6.09
97898,ORDER_87975,3,PRODUCT_28697,SELLER_0417,49.9,24.37
97899,ORDER_87975,4,PRODUCT_28697,SELLER_0417,49.9,24.37
97900,ORDER_87975,5,PRODUCT_28697,SELLER_0417,49.9,24.37


In [106]:
orders.loc[orders['Order_id']=='ORDER_87975']

,Order_id,Customer_id,Order_status,Order_purchase_timestamp,Order_delivered_carrier_date,Order_delivered_customer_date,Order_estimated_delivery_date
87842,ORDER_87975,CUSTOMER_19260,delivered,2019-06-05 12:28:11,2019-06-06 13:12:00,2019-06-07 17:28:31,2019-07-11


In [102]:
reviews.loc[reviews['Order_id']=='ORDER_87975']

,Review_id,Order_id,Review_score,Review_creation_date,Review_answer_timestamp
48859,REVIEW_48713,ORDER_87975,5,2019-06-08 00:00:00,2019-06-09 02:17:23


## 아니 한 주문에 서로 다른 거 여러 개 구매했는데 리뷰는 하나로 끝남... ㅋ

동일 주문(오더 아이디) 내에 같은 제품(프로덕트 아이디)을 여러 개 담은 경우 리뷰는 1개로 통합하고,  
동일 주문 내에 여러 제품을 담은 경우 각 제품마다 해당 점수를 부여 ...

reviews에서 오더 아이디랑 프로덕트 아이디랑 동일할 경우에 합치게 하고 리뷰 점수는 평균으로 하게 ...(그럼 걍 원 점수 들어감)  
이런 식이면 리뷰즈에 제품을 붙일 수가 없음  
차라리 오더즈에 점수를 넣는 방법임

In [103]:
reviews

,Review_id,Order_id,Review_score,Review_creation_date,Review_answer_timestamp
0,REVIEW_00000,ORDER_01674,4,2019-01-18 00:00:00,2019-01-18 21:46:59
1,REVIEW_00001,ORDER_80140,5,2019-03-10 00:00:00,2019-03-11 03:05:13
2,REVIEW_00002,ORDER_69816,5,2019-02-17 00:00:00,2019-02-18 14:36:24
3,REVIEW_00003,ORDER_24398,5,2018-04-21 00:00:00,2018-04-21 22:02:06
4,REVIEW_00004,ORDER_70366,5,2019-03-01 00:00:00,2019-03-02 10:26:53
...,...,...,...,...,...
87868,REVIEW_87298,ORDER_58840,5,2019-07-07 00:00:00,2019-07-14 17:18:30
87869,REVIEW_87299,ORDER_75162,5,2018-12-09 00:00:00,2018-12-11 20:06:42
87870,REVIEW_87300,ORDER_08690,5,2019-03-22 00:00:00,2019-03-23 09:10:43
87871,REVIEW_87301,ORDER_25681,4,2019-07-01 00:00:00,2019-07-02 12:59:13


# 춘식님 요청

In [112]:
df[df.duplicated(['Order_id'])]

,Order_id,Product_id,Order_item_id,Seller_id,Price,Freight_value
23,ORDER_00022,PRODUCT_12896,2,SELLER_0823,39.50,48.64
58,ORDER_00056,PRODUCT_07372,1,SELLER_2445,119.00,24.97
108,ORDER_00105,PRODUCT_14388,2,SELLER_1059,119.99,28.05
109,ORDER_00105,PRODUCT_16209,1,SELLER_1678,179.90,8.41
124,ORDER_00119,PRODUCT_23470,5,SELLER_0613SELLER_0613,183.98,27.00
...,...,...,...,...,...,...
91092,ORDER_87885,PRODUCT_11016,2,SELLER_1436,180.00,0.29
91119,ORDER_87912,PRODUCT_10235,1,SELLER_0062,209.99,15.90
91183,ORDER_87975,PRODUCT_28697,13,SELLER_0417SELLER_0417SELLER_0417SELLER_0417,199.60,97.48
91231,ORDER_88022,PRODUCT_16854,1,SELLER_0730,28.50,5.01


In [120]:
df2 = order_items.groupby(['Order_id', 'Product_id', 'Seller_id']).sum().reset_index()

In [122]:
df2[df2.duplicated(['Order_id'])]

,Order_id,Product_id,Seller_id,Order_item_id,Price,Freight_value
23,ORDER_00022,PRODUCT_12896,SELLER_0823,2,39.50,48.64
58,ORDER_00056,PRODUCT_07372,SELLER_2445,1,119.00,24.97
108,ORDER_00105,PRODUCT_14388,SELLER_1059,2,119.99,28.05
109,ORDER_00105,PRODUCT_16209,SELLER_1678,1,179.90,8.41
124,ORDER_00119,PRODUCT_23470,SELLER_0613,5,183.98,27.00
...,...,...,...,...,...,...
91092,ORDER_87885,PRODUCT_11016,SELLER_1436,2,180.00,0.29
91119,ORDER_87912,PRODUCT_10235,SELLER_0062,1,209.99,15.90
91183,ORDER_87975,PRODUCT_28697,SELLER_0417,13,199.60,97.48
91231,ORDER_88022,PRODUCT_16854,SELLER_0730,1,28.50,5.01


In [129]:
df2['Order_id'].value_counts()

Order_id
ORDER_61909    7
ORDER_03933    7
ORDER_46263    6
ORDER_82639    6
ORDER_26616    6
              ..
ORDER_29848    1
ORDER_29847    1
ORDER_29846    1
ORDER_29845    1
ORDER_29855    1
Name: count, Length: 87954, dtype: int64

In [152]:
df2['Order_id'].apply(lambda x : len(x)).max()

11

In [149]:
df2[df2['Order_id'].apply(lambda x : True if len(x) > 11 else False)]

,Order_id,Product_id,Seller_id,Order_item_id,Price,Freight_value


In [134]:
df2['Seller_id'][0].__len__()

11

In [146]:
df2[df2['Seller_id'].__len__() > 12]

KeyError: True

In [108]:
order_items.loc[order_items['Order_id']=='ORDER_00119']

,Order_id,Order_item_id,Product_id,Seller_id,Price,Freight_value
95656,ORDER_00119,1,PRODUCT_01292,SELLER_0613,91.99,6.75
95657,ORDER_00119,2,PRODUCT_23470,SELLER_0613,91.99,13.50
95658,ORDER_00119,3,PRODUCT_23470,SELLER_0613,91.99,13.50


# ㅇㅇㅇ

In [161]:
order_items

,Order_id,Order_item_id,Product_id,Seller_id,Price,Freight_value
0,ORDER_75477,1,PRODUCT_23128,SELLER_0452,58.90,13.29
1,ORDER_63604,1,PRODUCT_24351,SELLER_0414,239.90,19.93
2,ORDER_05566,1,PRODUCT_20226,SELLER_1637,199.00,17.87
3,ORDER_04632,1,PRODUCT_07878,SELLER_1445,199.90,18.14
4,ORDER_61452,1,PRODUCT_03505,SELLER_0581,21.90,12.69
...,...,...,...,...,...,...
100552,ORDER_70405,1,PRODUCT_04209,SELLER_2078,299.99,43.41
100553,ORDER_62069,1,PRODUCT_08229,SELLER_1593,350.00,36.53
100554,ORDER_46632,1,PRODUCT_21462,SELLER_0528,99.90,16.95
100555,ORDER_52966,1,PRODUCT_19978,SELLER_2563,55.99,8.72


full outer join
